In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [53]:
from sklearn.datasets import load_svmlight_files
import urllib.request

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import cvxopt
from sklearn.svm import SVC

In [3]:
# Download dataset
url = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/satimage.scale.tr"
urllib.request.urlretrieve(url, "satimage.scale.tr")

# Load dataset
X_train, y_train = load_svmlight_files(['satimage.scale.tr'])
print(X_train.shape)
print(y_train.shape)

(3104, 36)
(3104,)


In [4]:
# Download dataset
url2 = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/satimage.scale.t"
urllib.request.urlretrieve(url2, "satimage.scale.t")

# Load dataset
X_test, y_test = load_svmlight_files(['satimage.scale.t'])
print(X_test.shape)
print(y_test.shape)

(2000, 36)
(2000,)


In [5]:
# Download dataset
url3 = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/satimage.scale.val"
urllib.request.urlretrieve(url3, "satimage.scale.val")

# Load dataset
X_val, y_val = load_svmlight_files(['satimage.scale.val'])
print(X_val.shape)
print(y_val.shape)

(1331, 36)
(1331,)


In [6]:
type(X_train) , type(y_train)

(scipy.sparse._csr.csr_matrix, numpy.ndarray)

In [7]:
X_train[0][0]

<1x36 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>

In [8]:
# Classes to keep
classes = [4, 6]

# Filter rows by y_train values
keep = np.where(np.isin(y_train, classes))[0]

# Apply indices to sparse X_train
X_filtered = X_train[keep, :]

# Apply indices to dense y_train
y_filtered = y_train[keep]
print(X_filtered.shape)
print(y_filtered.shape)

(847, 36)
(847,)


In [9]:
type(y_filtered)

numpy.ndarray

In [10]:
type(X_filtered)

scipy.sparse._csr.csr_matrix

In [11]:
y_filtered[250:350]

array([6., 6., 6., 6., 4., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 4., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 4., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 4.,
       6., 6., 4., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       4., 6., 4., 6., 4., 6., 6., 6., 4., 6., 6., 6., 6., 6., 6.])

In [12]:
y_binary = np.where(y_filtered == 4, -1, np.where(y_filtered == 6, 1, y_filtered))
y_binary[250:350]

array([ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,
        1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [13]:

y = y_binary
X = X_filtered
# Parameters
C = 1.0
X = X.toarray()

# Kernel matrix
K = X @ X.T


In [14]:
# Construct QP

P = cvxopt.matrix(np.outer(y, y) * K)
q = cvxopt.matrix(-np.ones((len(y), 1)))

#G = cvxopt.matrix(np.diag(np.ones(len(y)) * -1))
top = np.diag(np.ones(len(y)) * -1)
bot = np.identity(len(y))
G = cvxopt.matrix(np.vstack((top, bot)))


left = np.zeros(len(y))
right  = np.ones(len(y)) * C
h = cvxopt.matrix(np.hstack((left,right)))

A = cvxopt.matrix(y, (1,len(y)))
b = cvxopt.matrix(np.zeros(1))


In [15]:
print(P.size, q.size, G.size, h.size, A.size, b.size)


(847, 847) (847, 1) (1694, 847) (1694, 1) (1, 847) (1, 1)


In [16]:
# Solve QP model
sol = cvxopt.solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0: -4.1814e+02 -2.1475e+03  1e+04  3e+00  1e-13
 1: -2.8867e+02 -1.4286e+03  2e+03  3e-01  1e-13
 2: -2.7710e+02 -4.7698e+02  2e+02  2e-02  9e-14
 3: -3.0754e+02 -3.9958e+02  1e+02  8e-03  9e-14
 4: -3.2114e+02 -3.7229e+02  5e+01  4e-03  1e-13
 5: -3.2791e+02 -3.6008e+02  3e+01  2e-03  9e-14
 6: -3.3417e+02 -3.4949e+02  2e+01  9e-04  1e-13
 7: -3.3732e+02 -3.4436e+02  7e+00  3e-04  1e-13
 8: -3.3870e+02 -3.4213e+02  3e+00  1e-04  1e-13
 9: -3.3959e+02 -3.4086e+02  1e+00  3e-05  1e-13
10: -3.4009e+02 -3.4022e+02  1e-01  6e-14  1e-13
11: -3.4015e+02 -3.4016e+02  8e-03  6e-14  1e-13
12: -3.4016e+02 -3.4016e+02  2e-04  1e-13  1e-13
Optimal solution found.


In [17]:

# Lagrange multipliers
alphas = np.ravel(sol['x'])

# Extract support vectors
sv = alphas > 1e-5
ind = np.arange(len(alphas))[sv]

# Dual solution
alpha_sv = alphas[sv]
SV = X_filtered[sv]
y_sv = y[sv]

print("Alphas = ", alpha_sv)
print("Support vectors = ", SV)
print("y support vectors = ", y_sv)



Alphas =  [9.99997832e-01 9.99999950e-01 9.99999024e-01 9.99999270e-01
 9.99999767e-01 9.99999959e-01 9.99999864e-01 7.03007394e-01
 9.99999954e-01 9.99999932e-01 9.99999915e-01 9.99999849e-01
 3.66964216e-01 9.99999934e-01 9.99999914e-01 9.99999869e-01
 9.99999798e-01 9.99999949e-01 9.99999938e-01 9.99999958e-01
 9.99999932e-01 9.99999960e-01 9.99999959e-01 9.99999930e-01
 9.99999938e-01 9.99999939e-01 9.99999917e-01 9.99999855e-01
 9.99999888e-01 9.99999918e-01 9.99999874e-01 9.99999926e-01
 9.99999957e-01 9.99999950e-01 9.99999910e-01 9.92079780e-01
 9.99999667e-01 9.99999798e-01 9.99999915e-01 9.99999953e-01
 9.99999915e-01 9.99999857e-01 9.99999939e-01 9.99999878e-01
 9.99999892e-01 9.99999801e-01 9.99999752e-01 9.99999922e-01
 9.99999949e-01 9.99999860e-01 9.99999932e-01 9.99999699e-01
 4.00327333e-02 9.99999945e-01 9.99999696e-01 1.33471266e-01
 9.99999870e-01 9.99999892e-01 9.99999922e-01 9.99999645e-01
 9.99999380e-01 9.99999859e-01 9.99999906e-01 9.99999859e-01
 9.99999945e-0

In [18]:
len(y_sv) , alphas[ : 25]

(369,
 array([9.99997832e-01, 6.88198407e-08, 9.99999950e-01, 1.50609186e-07,
        9.99999024e-01, 1.25288520e-06, 9.99999270e-01, 9.99999767e-01,
        1.30979563e-06, 9.99999959e-01, 9.99999864e-01, 4.47877665e-07,
        7.03007394e-01, 9.99999954e-01, 6.73799816e-07, 9.99999932e-01,
        9.99999915e-01, 9.99999849e-01, 3.66964216e-01, 9.99999934e-01,
        9.99999914e-01, 2.61362686e-07, 9.99999869e-01, 9.99999798e-01,
        9.99999949e-01]))

In [19]:
# Classes to keep
classes = [4, 6]

# Filter rows by y_train values
keep = np.where(np.isin(y_test, classes))[0]

# Apply indices to sparse X_train
X_test_filtered = X_test[keep, :]
X_test_filtered_duplicate = X_test_filtered

# Apply indices to dense y_train
y_test_filtered = y_test[keep]
print(X_test_filtered.shape)
print(y_test_filtered.shape)

(681, 36)
(681,)


In [20]:
type(X_test_filtered)

scipy.sparse._csr.csr_matrix

In [21]:
X_test_filtered = X_test_filtered.toarray()

In [22]:
type(X_test_filtered)

numpy.ndarray

In [23]:
y_test_filtered[100:150]

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [24]:
y_test_binary = np.where(y_test_filtered == 4, -1, np.where(y_test_filtered == 6, 1, y_test_filtered))
y_test_binary[250:350]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
K[1,1]

5.65923919066772

In [26]:
b = 0
for n in range(len(alpha_sv)):
  b += y_sv[n]
  b -= sum(alpha_sv * y_sv * K[ind[n],sv])
  b /= len(alpha_sv)

w = np.zeros(len(X[0]))
for n in range(len(alpha_sv)):
  w += alpha_sv[n] * y_sv[n] * SV[n]


In [27]:
w

matrix([[ 0.33570497, -0.67034467, -1.65974   , -0.75634827,  1.12765625,
         -0.50353652,  0.02170402,  1.03046887,  0.92544067,  0.72951705,
         -0.45742565, -0.58279165,  0.04666628, -0.26809819, -0.27229659,
          0.46147562, -1.4048444 , -2.12598027, -0.56439241, -0.38525066,
         -0.02085424, -1.69192637, -0.55541202,  0.40688559,  0.39689489,
         -1.38057235, -0.46925903,  0.57136415,  0.9816216 , -0.72986015,
         -0.64655624,  0.17091224,  1.48801066,  0.19266652, -0.84041541,
         -0.1402692 ]])

In [28]:
b

-0.0013955176573914683

In [29]:
def svm_predict(X_test):
    return np.sign(w * X_test.T + b)

In [30]:
# Make predictions on test set
y_pred = svm_predict(X_test_filtered)

In [31]:
y_test_filtered.shape
type(y_test_filtered)

numpy.ndarray

In [32]:
type(y_pred)

numpy.matrix

In [33]:
y_pred = np.squeeze(np.asarray(y_pred))

In [34]:
y_pred.shape

(681,)

In [35]:
# Accuracy
def accuracy_(y_true, y_pred):
  correct = np.sum(y_true == y_pred)
  total = len(y_true)
  return correct / total

# Confusion matrix
def confusion_matrix_(y_true, y_pred):
  unique = np.unique(y_true)
  matrix = np.zeros((len(unique), len(unique)))
  for i, t in enumerate(y_true):
    j = np.argmax(y_pred[i])
    matrix[t][j] += 1
  return matrix

# Balanced accuracy
def balanced_accuracy_(y_true, y_pred):
  unique, counts = np.unique(y_true, return_counts=True)
  class_weights = counts / len(y_true)
  scores = [accuracy(y_true[y_true==label], y_pred[y_true==label]) for label in unique]
  return np.average(scores, weights=class_weights)

In [36]:

# Evaluate metrics
accuracy = accuracy_score(y_test_binary, y_pred)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_test_binary, y_pred)
print("Confusion Matrix:\n", conf_matrix)

bal_accuracy = balanced_accuracy_score(y_test_binary, y_pred)
print("Balanced Accuracy:", bal_accuracy)

Accuracy: 0.8120411160058737
Confusion Matrix:
 [[157  54]
 [ 74 396]]
Balanced Accuracy: 0.7933145104366239


**Implementing the RBF Kernel**

In [37]:
classes = [4, 6]
keep = np.where(np.isin(y_val, classes))[0]
X_val_filtered = X_val[keep, :]
y_val_filtered = y_val[keep]
y_val_binary = np.where(y_val_filtered == 4, -1, np.where(y_val_filtered == 6, 1, y_val_filtered))
print(X_val_filtered.shape)
print(y_val_binary.shape)
type(X_val_filtered),type(y_val_binary)

(606, 36)
(606,)


(scipy.sparse._csr.csr_matrix, numpy.ndarray)

In [38]:
def RBF(x , y, gamma=1):
    return np.exp(-gamma * np.linalg.norm(x - y, ord=2))

In [39]:
def train_svm(inpute_data, label, parameter , Kernel ):
  y = label
  X = inpute_data
  C = parameter
  X = X.toarray()
  # Kernel matrix
  K = X @ X.T
  P = cvxopt.matrix(np.outer(y, y) * K)
  q = cvxopt.matrix(-np.ones((len(y), 1)))
  top = np.diag(np.ones(len(y)) * -1)
  bot = np.identity(len(y))
  G = cvxopt.matrix(np.vstack((top, bot)))
  left = np.zeros(len(y))
  right  = np.ones(len(y)) * C
  h = cvxopt.matrix(np.hstack((left,right)))
  A = cvxopt.matrix(y, (1,len(y)))
  b = cvxopt.matrix(np.zeros(1))
  sol = cvxopt.solvers.qp(P, q, G, h, A, b)
  alphas = np.ravel(sol['x'])
  sv = alphas > 1e-5
  ind = np.arange(len(alphas))[sv]
  alpha_sv = alphas[sv]
  SV = X_filtered[sv]
  y_sv = y[sv]
  b = 0
  for n in range(len(alpha_sv)):
    b += y_sv[n]
    b -= sum(alpha_sv * y_sv * K[ind[n],sv])
    b /= len(alpha_sv)

  return alpha_sv, SV, y_sv, b

In [40]:
def svm_prediction_kernel(x, kernel, a, sv, sv_y, b , g):
  x = x.toarray()
  y_predict = np.zeros(len(x))
  for i in range(len(x)):
    s = 0
    for _a, _sv_y, _sv in zip(a, sv_y, sv):
      s += _a * _sv_y * kernel(x[i], _sv , gamma=g)
    y_predict[i] = s
  p = y_predict + b
  return np.sign(p)


In [41]:
gammas = [0.01, 0.1, 0.2 ,0.5, 1, 10, 100]
for _gamma in gammas:
  print(f'the gamma value is :  {_gamma}')
  a1, sv1, sv_y1, b1 = train_svm(X_filtered, y_binary , 1.0 , RBF)
  y_val_pred = svm_prediction_kernel(X_val_filtered, RBF, a1, sv1, sv_y1, b1 ,_gamma)
  y_val_pred = np.squeeze(np.asarray(y_val_pred))
  accuracy = accuracy_score(y_val_binary, y_val_pred)
  print("Accuracy:", accuracy)
  conf_matrix = confusion_matrix(y_val_binary, y_val_pred)
  print("Confusion Matrix:\n", conf_matrix)
  bal_accuracy = balanced_accuracy_score(y_val_binary, y_val_pred)
  print("Balanced Accuracy:", bal_accuracy)

the gamma value is :  0.01
     pcost       dcost       gap    pres   dres
 0: -4.1814e+02 -2.1475e+03  1e+04  3e+00  1e-13
 1: -2.8867e+02 -1.4286e+03  2e+03  3e-01  1e-13
 2: -2.7710e+02 -4.7698e+02  2e+02  2e-02  9e-14
 3: -3.0754e+02 -3.9958e+02  1e+02  8e-03  9e-14
 4: -3.2114e+02 -3.7229e+02  5e+01  4e-03  1e-13
 5: -3.2791e+02 -3.6008e+02  3e+01  2e-03  9e-14
 6: -3.3417e+02 -3.4949e+02  2e+01  9e-04  1e-13
 7: -3.3732e+02 -3.4436e+02  7e+00  3e-04  1e-13
 8: -3.3870e+02 -3.4213e+02  3e+00  1e-04  1e-13
 9: -3.3959e+02 -3.4086e+02  1e+00  3e-05  1e-13
10: -3.4009e+02 -3.4022e+02  1e-01  6e-14  1e-13
11: -3.4015e+02 -3.4016e+02  8e-03  6e-14  1e-13
12: -3.4016e+02 -3.4016e+02  2e-04  1e-13  1e-13
Optimal solution found.
Accuracy: 0.17986798679867988
Confusion Matrix:
 [[109   0]
 [497   0]]
Balanced Accuracy: 0.5
the gamma value is :  0.1
     pcost       dcost       gap    pres   dres
 0: -4.1814e+02 -2.1475e+03  1e+04  3e+00  1e-13
 1: -2.8867e+02 -1.4286e+03  2e+03  3e-01  1e-

In [42]:
gamma_ = 1
sigma = np.sqrt(1 / (2 * gamma_))
print("the best value for gamma is:\n", gamma_)
print("the best value for sigam is :\n", sigma)

the best value for gamma is:
 1
the best value for sigam is :
 0.7071067811865476


In [43]:
b

-0.0013955176573914683

In [44]:
y_test_pred_kernel = svm_prediction_kernel(X_test_filtered_duplicate, RBF, alpha_sv, SV, y_sv, b ,gamma_)
y_test_pred_kernel = np.squeeze(np.asarray(y_test_pred_kernel))
accuracy = accuracy_score(y_test_binary, y_test_pred_kernel)
print("Accuracy:", accuracy)
conf_matrix = confusion_matrix(y_test_binary, y_test_pred_kernel)
print("Confusion Matrix:\n", conf_matrix)
bal_accuracy = balanced_accuracy_score(y_test_binary, y_test_pred_kernel)
print("Balanced Accuracy:", bal_accuracy)

Accuracy: 0.6211453744493393
Confusion Matrix:
 [[211   0]
 [258 212]]
Balanced Accuracy: 0.725531914893617


Unfortunately it did not improve

**SVM With Scikit Learn**

In [56]:
acc_best , conf_mat_best , blance_acc_best = 0, 0, 0
C_R = np.logspace(-3, 6, 20)
gamma_R = np.logspace(-6, 3, 10)
for _ in C_R:
    for g1 in gamma_R:
        model = SVC(C=_, gamma=g1)
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        acc = balanced_accuracy_score(y_test, y_predict)
        conf_mat = confusion_matrix(y_test, y_predict)
        blance_acc = balanced_accuracy_score(y_test, y_predict)
        if blance_acc > blance_acc_best:
          acc_best = acc
          conf_mat_best = conf_mat
          blance_acc_best = blance_acc
          print('######################################')
          print('found a better balanced accuracy')
          print("the C value:", _)
          print("the gamma value:", g1)
          print("Accuracy:", acc_best)
          print("Confusion Matrix:\n", conf_mat_best)
          print("Balanced Accuracy:", blance_acc_best)
          print('######################################')

print("\n")
print('######################################')
print("Accuracy:", acc_best)
print("Confusion Matrix:\n", conf_mat_best)
print("Balanced Accuracy:", blance_acc_best)

######################################
found a better balanced accuracy
the C value: 0.001
the gamma value: 1e-06
Accuracy: 0.16666666666666666
Confusion Matrix:
 [[  0   0 461   0   0   0]
 [  0   0 224   0   0   0]
 [  0   0 397   0   0   0]
 [  0   0 211   0   0   0]
 [  0   0 237   0   0   0]
 [  0   0 470   0   0   0]]
Balanced Accuracy: 0.16666666666666666
######################################
######################################
found a better balanced accuracy
the C value: 0.002976351441631319
the gamma value: 0.1
Accuracy: 0.510539306830854
Confusion Matrix:
 [[439   0  22   0   0   0]
 [ 77 147   0   0   0   0]
 [  7   0 390   0   0   0]
 [107   0 103   0   0   1]
 [226   0   3   0   0   8]
 [192   0  56   0   0 222]]
Balanced Accuracy: 0.510539306830854
######################################
######################################
found a better balanced accuracy
the C value: 0.008858667904100823
the gamma value: 0.1
Accuracy: 0.6243995262890162
Confusion Matrix:
 [[453   

**Multi Class Classifier for SVM**

In [58]:
print(X_train.shape)
print(y_train.shape)

(3104, 36)
(3104,)


In [64]:
# Classes to keep
classes = [1, 3, 6]
keep = np.where(np.isin(y_train, classes))[0]

# Apply indices to sparse X_train
X_filtered_multi = X_train[keep, :]
X_filtered_multi = X_filtered_multi.toarray()
# Apply indices to dense y_train
y_filtered_multi = y_train[keep]
print(X_filtered_multi.shape)
print(y_filtered_multi.shape)


(2120, 36)
(2120,)


In [60]:
print(X_test.shape)
print(y_test.shape)

(2000, 36)
(2000,)


In [76]:
classes = [1, 3, 6]

keep2 = np.where(np.isin(y_test, classes))[0]

# Apply indices to sparse X_train
X_test_filtered_multi = X_test[keep2, :]
X_test_filtered_multi = X_test_filtered_multi.toarray()
# Apply indices to dense y_train
y_test_filtered_multi = y_test[keep2]

print(X_test_filtered_multi.shape)
print(y_test_filtered_multi.shape)

(1328, 36)
(1328,)


In [82]:
n_classes = np.unique(y_test_filtered_multi).shape[0]
n_classes

3

In [92]:
import numpy as np

class SVM:
    def __init__(self, C=1.0):
        self.C = C
        self.W = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.W = np.zeros((n_classes, n_features))

        gram = self.kernel(X, X)
        print(gram)
        for c in range(n_classes):

            indices = np.where(y == c)[0]
            Xc = X[indices,:]
            yc = y[indices]
            dual_coef = self.solve_dual(gram[np.ix_(indices, indices)], yc)
            self.W[c,:] = np.dot(dual_coef, Xc)

    def predict(self, X):
        y_pred = []
        for x in X:
            scores = np.dot(self.W, x)

            y_pred.append(np.argmax(scores))
        return np.array(y_pred)

    def kernel(self, X1, X2):
        return np.dot(X1, X2.T)

    def solve_dual(self, K, y):
        n_samples = len(y)
        alpha = np.zeros(n_samples)
        steps = 100
        C = self.C

        for i in range(steps):
            h = np.dot(K, alpha)
            E = h - y

            mask = np.logical_and(alpha > 0, alpha < C)
            alpha[mask] -= (E[mask] / K.diagonal()[mask])

            dual_obj = alpha.T @ h - 0.5 * alpha.T @ K @ alpha - y.T @ alpha
            primal_obj = 0.5 * np.linalg.norm(self.W) ** 2 + self.C * np.sum(np.maximum(0, 1 - y * E))
            gap = primal_obj - dual_obj
            if gap < 1e-5:
                break

        return alpha

In [93]:
svm = SVM(C=1.0)
svm.fit(X_filtered_multi, y_filtered_multi)

[[ 3.32244512  3.99224476  1.05431032 ... -0.99821331 -0.64836757
  -1.67065703]
 [ 3.99224476  5.65923919  1.13992337 ... -1.48850685 -0.92960544
  -2.66733434]
 [ 1.05431032  1.13992337  3.00325734 ...  2.62193036  1.93345695
   3.11732325]
 ...
 [-0.99821331 -1.48850685  2.62193036 ...  5.57386133  3.86634753
   6.19019946]
 [-0.64836757 -0.92960544  1.93345695 ...  3.86634753  2.94749308
   4.36553571]
 [-1.67065703 -2.66733434  3.11732325 ...  6.19019946  4.36553571
   7.58666583]]


In [94]:
y_pred = svm.predict(X_test_filtered_multi)

In [95]:
acc = np.mean(y_pred == y_filtered_multi)

<ipython-input-95-e9b4553ebc13>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  acc = np.mean(y_pred == y_filtered_multi)


In [96]:
acc

0.0